In [1]:
# Helper Function to Create Prompt in Bucket

from minio import Minio
from io import BytesIO

def create_prompt_in_minio(client, bucket_name, object_name, prompt_content):
    """
    Create a text file and upload it to a specified MinIO bucket.

    :param client: Minio client instance
    :param bucket_name: Name of the MinIO bucket
    :param object_name: Object name in the bucket (e.g., 'rag-prompt.txt')
    :param prompt_content: Content to write into the prompt file
    """
    try:
        content_bytes = prompt_content.encode('utf-8')  # Convert the string content to bytes
        content_length = len(content_bytes)
        content_stream = BytesIO(content_bytes)
        # Put object in the specified bucket
        client.put_object(bucket_name, object_name, content_stream, content_length)
        print(f"Prompt '{object_name}' created and uploaded successfully in bucket '{bucket_name}'.")
    except Exception as e:
        print(f"Failed to upload prompt: {e}")

# Usage
minio_client = Minio(
    "play.min.io:443",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=True
)

prompt_content = "Describe the latest trends in AI and their implications for future technology."
create_prompt_in_minio(minio_client, "prompt-bucket", "rag-prompt.txt", prompt_content)

Prompt 'rag-prompt.txt' created and uploaded successfully in bucket 'prompt-bucket'.


In [2]:
# Helper Function to Setup Weavaite
import weaviate
import logging

def setup_weaviate_for_langchain(base_url):
    """
    Initialize a Weaviate client and create a class for use with LangChain.

    :param base_url: URL to the Weaviate instance.
    :return: Initialized Weaviate client or None if initialization fails.
    """
    try:
        client = weaviate.Client(base_url)
        schema = {
            "classes": [
                {
                    "class": "TextData",
                    "description": "Store text data for RAG processing",
                    "properties": [
                        {
                            "name": "text",
                            "dataType": ["text"],
                            "description": "The content of the text",
                        },
                    ],
                },
            ]
        }
        # Create the schema in Weaviate
        client.schema.create(schema)
        logging.info("Schema created successfully.")
        return client
    except Exception as e:
        logging.error(f"Failed to initialize Weaviate client or create schema: {e}")
        return None

# Example usage
weaviate_url = "https://minio-rag-kexh68uw.weaviate.network"
weaviate_client = setup_weaviate_for_langchain(weaviate_url)

if weaviate_client:
    print("Weaviate client is set up and schema is ready to use.")
else:
    print

/var/mobile/Containers/Data/Application/363B3AE4-B614-4655-8E4D-607A30FAD940/Library/Application Support/com.rationalmatter.junoapp/python-home/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Weaviate client is set up and schema is ready to use.


In [2]:
# Function to retreive rag-prompt.txt from prompt-bucket

from minio import Minio
from io import BytesIO

def get_prompt_from_minio(client, bucket_name, object_name):
    """
    Retrieve a text file from a specified MinIO bucket and return its content as a string.

    :param client: Minio client instance
    :param bucket_name: Name of the MinIO bucket
    :param object_name: Object name in the bucket (e.g., 'rag-prompt.txt')
    :return: Content of the prompt as a string
    """
    try:
        # Get object data from the specified bucket
        response = client.get_object(bucket_name, object_name)
        data = response.read()  # Read data from the response
        prompt_content = data.decode('utf-8')  # Decode bytes to string
        return prompt_content
    except Exception as e:
        print(f"Failed to retrieve prompt: {e}")
        return None

# Usage
minio_client = Minio(
    "play.min.io:443",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=True
)

prompt_content = get_prompt_from_minio(minio_client, "prompt-bucket", "rag-prompt.txt")
if prompt_content:
    print(f"Retrieved prompt content: {prompt_content}")
else:
    print("No content retrieved.")

Retrieved prompt content: Describe the latest trends in AI and their implications for future technology.
